In [ ]:
### Autodock preparation ### from: https://www.chem.uwec.edu/chem491_w01/Chem491-Molecules%20and%20Medicine%202008/AutoDock%20Tutorial.pdf

In [ ]:


from SBLMDCOVDOCK.SBLSettings import DOCKING_Settings
from SBLMDCOVDOCK.md_setuptools import plot_xvg
import pickle
from copy import deepcopy

import pandas as pd
import os
import glob

# import gmxapi as gmx
import subprocess
import matplotlib.pyplot as plt

settings = DOCKING_Settings()
main_dir = os.getcwd()

# AMPC - https://journals.asm.org/doi/10.1128/AAC.02073-20
ampc = "6T3D"
# KPC-2 - BLDB: http://dx.doi.org/10.1021/ACS.JMEDCHEM.7B00158
kpc2 = "5UL8"
# OXA-10 - BLDB: https://www.pnas.org/doi/full/10.1073/pnas.241442898
oxa10 = "1K55"

structures = pd.DataFrame({"PDBID": [ampc, kpc2, oxa10], 
                           "Name": ["AmpC", "KPC2", "OXA10"]})



In [ ]:
prepare_ligand_path = os.path.join(settings.structures_input, "prepare_ligand4.py")
print(prepare_ligand_path)
print(os.path.exists(prepare_ligand_path))
print(main_dir)
os.chdir(main_dir)

In [ ]:
ligands = os.listdir(settings.structures_input)
ligands = [ligand for ligand in ligands if ligand.endswith(".sdf")]
print(ligands)

In [ ]:
import subprocess
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"
# os.chdir(settings.structures_input)

for idx, lig in enumerate(ligands):
    lig_pdb = lig.replace(".sdf", ".pdb")
    print(idx, lig, lig_pdb)
    obabel_command = ["obabel", 
                      "-i", "sdf", 
                      "-o", "pdb", lig, 
                      "-O", lig_pdb, 
                    #   "-h"
                      ]

    subprocess.run(obabel_command, 
                       check=True,
                       cwd=settings.structures_input)

    # break
# os.chdir(main_dir)


In [ ]:
import subprocess
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"
# os.chdir(settings.structures_input)
ligs = []
for idx, lig in enumerate(ligands):
    lig_pdb = lig.replace(".sdf", ".pdb")

    script = "prepare_ligand4.py"

    lig_out = lig_pdb.replace(".pdb", ".pdbqt")

    lig_pdb = os.path.join(settings.structures_input, lig_pdb)
    lig_out = os.path.join(settings.topology,settings.parent, lig_out)    

    print(idx, lig_pdb, lig_out)
    script_path = os.path.join(settings.structures_input, script)
    preplig4_command = [pythonsh, script_path, 
                        "-l", lig_pdb, 
                        "-o", lig_out,
                        "-A", "bonds_hydrogens",
                        "-U", "nphs_lps",
                        "-v"]
    try:
        subprocess.run(preplig4_command, 
                       check=True)
        ligs.append(lig.replace(".sdf", ""))
    except:
        pass

    # break
# os.chdir(main_dir)


In [ ]:
print(ligs)

In [ ]:
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"

for lig in ligs:
    for pdb in structures.PDBID:
        break
        print(os.getcwd())
        script_name = "prepare_gpf4.py"
        prepare_grid_path = os.path.join("..","..",os.path.join(settings.config, settings.parent, script_name))

        pdb_name = "APO_" +pdb + "_ADT" + ".pdb"
        pdbqt_name = pdb_name.replace(".pdb", ".pdbqt")
        ligand_name = lig + ".pdbqt"
        prep_grid_command = [pythonsh,
                            prepare_grid_path,
                            "-r", pdbqt_name,
                            "-l", ligand_name,
                            "-o", lig +"_"+ pdbqt_name.replace(".pdbqt", ".gpf"),
                            # "-p", "ligand_types='H,HD,HS,C,A,N,NA,NS,OA,OS,F,Mg,Mn,Zn,Ca,Fe,Cl,Br,I,SA,S,Se,P,GA,GC,GN,GO,GP,G'",
                            "-v"]
        
        print(" ".join(prep_grid_command))
        dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"

        subprocess.run(prep_grid_command, cwd=dir)


    # break

In [ ]:
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"

for lig in ligs:
    for pdb in structures.PDBID:

        print(os.getcwd())
        script_name = "prepare_dpf4.py"
        prepare_dock_path = os.path.join("..","..",os.path.join(settings.config, settings.parent, script_name))

        pdb_name = "APO_" +pdb + "_ADT" + ".pdb"
        pdbqt_name = pdb_name.replace(".pdb", ".pdbqt")
        ligand_name = lig + ".pdbqt"
        prep_dock_command = [pythonsh,
                            prepare_dock_path,
                            "-l", ligand_name,
                            "-r", pdbqt_name,
                            "-o", lig +"_"+ pdbqt_name.replace(".pdbqt", ".dpf"),
                            "-v"]

        print(" ".join(prep_dock_command))
        dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"

        subprocess.run(prep_dock_command, cwd=dir, check=True)




In [ ]:
print(os.getcwd())

In [ ]:
print(os.listdir(dir))

In [17]:

autodockgpu_dir = "/home/alexi/Documents/AutoDock-GPU/bin"
engine = "CUDA_64"

autodockgpu = f"autodock_{engine}wi"
auto_dock = os.path.join(autodockgpu_dir, autodockgpu)
docked = []
# for pdb in structures.PDBID:
pdb="5UL8"
for lig in ligs:
# for pdb in structures.PDBID:
    # print("Docking ", lig, " to ", pdb)
    name = "APO_" +pdb + "_ADT"
    pdb_name =  name + ".pdb"
    pdbqt_name = pdb_name.replace(".pdb", ".pdbqt")
    grid_name = lig +"_"+ pdbqt_name.replace(".pdbqt", ".gpf")
    grid_log = grid_name.replace(".gpf", ".glg")

    autogrid_command = ["autogrid4", 
                        "-p", grid_name, 
                        "-l", grid_log, 
                        "-d", f"{name}.maps.fld", 
                        "-o", f"{name}.maps.map",
                        "-v"]

    dock_name = lig +"_"+ pdbqt_name.replace(".pdbqt", ".dpf")
    dock_log = dock_name.replace(".dpf", ".dlg")

    autodock_command = [auto_dock, 
                        # "-p", dock_name, 
                        # "-l", dock_log, 
                        "-M",  f"{name}.maps.fld", 
                        "-L", f"{lig}.pdbqt",
                        "--lsmet", "ad",
                        # "--psize", "50",
                        # "--nrun", "50",
                        "-v"]


    dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"
    # print(os.listdir(dir))
    try:
        subprocess.run(autogrid_command, check=True,cwd=dir)
        subprocess.run(autodock_command, check=True,cwd=dir)
        print("Docking successful")
        docked.append((lig, pdb))
    except:
        pass
    # break
print(docked)


autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 307



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 308



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 590



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 591



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 598



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 598



autogrid4: WARNING:  Found an H-bonding

AutoDock-GPU version: v1.5.3-53-gf58a5173fa2f4c82a74b40bc895ad7e65de59f47

Running 1 docking calculation

Cuda device:                              NVIDIA GeForce RTX 3060
Available memory on device:               11076 MB (total: 12022 MB)

CUDA Setup time 0.048584s

Running Job #1
    Using heuristics: (capped) number of evaluations set to 1613968
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.003028s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 1613968 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples |    Best energy
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          150 |212852.61 kcal/mol |143755.91 +/-23032.23 kcal/mol |       5 |102798.97 kcal/mol
          5 |        29537 |212852.61 kcal/mol |


autogrid4: ERROR:  unknown ligand atom type B
add parameters for it to the parameter library first!



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 307



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 308



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 590



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 591



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 598



autogrid4: WARNING:  Found an H-

AutoDock-GPU version: v1.5.3-53-gf58a5173fa2f4c82a74b40bc895ad7e65de59f47

Running 1 docking calculation

Cuda device:                              NVIDIA GeForce RTX 3060
Available memory on device:               11076 MB (total: 12022 MB)

CUDA Setup time 0.033937s

Running Job #1
    Using heuristics: (capped) number of evaluations set to 1132076
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.002875s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 1132076 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples |    Best energy
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          150 |23861.10 kcal/mol |13967.36 +/- 2638.33 kcal/mol |       4 |10968.80 kcal/mol
          5 |        31984 |23861.10 kcal/mol | 443


autogrid4: ERROR:  unknown ligand atom type B
add parameters for it to the parameter library first!



autogrid4: ERROR:  unknown ligand atom type B
add parameters for it to the parameter library first!



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 307



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 308



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 590



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 591



autogrid4: WARNING:  Foun

...


autogrid4: ERROR:  unknown ligand atom type B
add parameters for it to the parameter library first!



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 307



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 308



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 590



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 591



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 598



autogrid4: WARNING:  Found an H-

AutoDock-GPU version: v1.5.3-53-gf58a5173fa2f4c82a74b40bc895ad7e65de59f47

Running 1 docking calculation

Cuda device:                              NVIDIA GeForce RTX 3060
Available memory on device:               11052 MB (total: 12022 MB)

CUDA Setup time 0.034869s

Running Job #1
    Using heuristics: (capped) number of evaluations set to 1132076
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.003455s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 1132076 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples |    Best energy
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          150 |21389.28 kcal/mol | 8436.48 +/- 3454.08 kcal/mol |       4 | 4647.44 kcal/mol
          5 |        31456 |21389.28 kcal/mol | 443


autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 307



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 308



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 311



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 312



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 590



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 591



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 598



autogrid4: WARNING:  Found an H-bonding atom with three bonded atoms, atom serial number 598



autogrid4: WARNING:  Found an H-bonding

AutoDock-GPU version: v1.5.3-53-gf58a5173fa2f4c82a74b40bc895ad7e65de59f47

Running 1 docking calculation

Cuda device:                              NVIDIA GeForce RTX 3060
Available memory on device:               11053 MB (total: 12022 MB)

CUDA Setup time 0.037466s

Running Job #1
    Using heuristics: (capped) number of evaluations set to 1132076
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.003062s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 1132076 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples |    Best energy
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          150 |24876.46 kcal/mol |10301.88 +/- 5040.84 kcal/mol |       6 | 2135.69 kcal/mol
          5 |        32455 |22903.99 kcal/mol | 321


autogrid4: ERROR:  unknown ligand atom type B
add parameters for it to the parameter library first!



autogrid4: ERROR:  unknown ligand atom type B
add parameters for it to the parameter library first!



autogrid4: ERROR:  unknown ligand atom type B
add parameters for it to the parameter library first!




In [16]:
print(docked)

[('NFF', '1K55'), ('XD1', '1K55'), ('ZX1', '1K55'), ('MXF', '1K55'), ('LCE', '1K55'), ('BIY', '1K55'), ('C9D', '1K55'), ('ZX0', '1K55'), ('NW3', '1K55'), ('SR3', '1K55'), ('TEM', '1K55'), ('4OV', '1K55'), ('MXS', '1K55'), ('CB9', '1K55'), ('1RG', '1K55'), ('5VR', '1K55'), ('JXG', '1K55'), ('DVR', '1K55'), ('AXL', '1K55'), ('LN1', '1K55'), ('X57', '1K55'), ('2AW', '1K55'), ('PNM', '1K55'), ('7EP', '1K55'), ('KCP', '1K55'), ('FOS', '1K55'), ('OP0', '1K55'), ('0RJ', '1K55'), ('9EP', '1K55'), ('17K', '1K55'), ('AIX', '1K55'), ('33V', '1K55'), ('CEP', '1K55'), ('CFX', '1K55'), ('4J6', '1K55'), ('HOQ', '1K55'), ('3GE', '1K55'), ('MK7', '1K55'), ('KCX', '1K55'), ('SFR', '1K55'), ('5VW', '1K55'), ('1X6', '1K55'), ('ISS', '1K55'), ('C8V', '1K55'), ('R0B', '1K55'), ('AP3', '1K55'), ('TJ7', '1K55'), ('ALP', '1K55'), ('MXC', '1K55'), ('SA2', '1K55'), ('CED', '1K55'), ('C8Y', '1K55'), ('S1Y', '1K55'), ('3S2', '1K55'), ('FK2', '1K55'), ('602', '1K55'), ('IPP', '1K55'), ('LK5', '1K55'), ('MER', '1K55